# Niger Phenolopy

## Setup

In [2]:
import sys, os
from os.path import join, expanduser, basename
import time
sys.path.append(join(expanduser('~'), 'Repos', 'GEE_Zonal', 'src'))

# Data
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import shapely
import geojson
import numpy as np

## Earth Engine APIs
import ee
from gee_zonal import Catalog, ZonalStats, authenticateGoogleDrive, gpd_to_gee
from geemap.conversion import *
from geemap import Map, colormaps

import xarray as xr
import rioxarray
from tqdm.notebook import tqdm
from dask.distributed import Client, LocalCluster
import requests

import matplotlib.pyplot as plt
from plotnine import *
from mizani.breaks import date_breaks
from mizani.formatters import date_format

from scipy.signal import savgol_filter, wiener
from scipy.stats import zscore
from statsmodels.tsa.seasonal import STL as stl

sys.path.append(join(expanduser('~'), 'Repos', 'phenolopy', 'scripts'))
import phenolopy
import dask

sys.path.append(join(expanduser('~'), 'Repos', 'GOSTrocks', 'src'))
import GOSTrocks.rasterMisc as rMisc

## Process Modis

In [8]:
start_period = ee.Date('2023-01-01')
end_period = ee.Date('2024-01-01')

terra = ee.ImageCollection('MODIS/061/MOD13Q1') \
    .select(['EVI','SummaryQA','DetailedQA'])   \
    .filterDate(start_period, end_period)
aqua = ee.ImageCollection('MODIS/061/MYD13Q1')  \
    .select(['EVI','SummaryQA','DetailedQA'])   \
    .filterDate(start_period, end_period)

modis = terra.select('EVI').merge(aqua.select('EVI'))
modis = modis.sort("system:time_start")

def bitwiseExtract(value, fromBit, toBit=None):
  if (toBit == None):
    toBit = fromBit
  maskSize = ee.Number(1).add(toBit).subtract(fromBit)
  mask = ee.Number(1).leftShift(maskSize).subtract(1)
  return value.rightShift(fromBit).bitwiseAnd(mask)

# Applying the SummaryQA  and DetailedQA
def modisQA_mask(image):
  sqa = image.select('SummaryQA')
  dqa = image.select('DetailedQA')
  viQualityFlagsS = bitwiseExtract(sqa, 0, 1)
  viQualityFlagsD = bitwiseExtract(dqa, 0, 1)
  # viUsefulnessFlagsD = bitwiseExtract(dqa, 2, 5)
  viSnowIceFlagsD = bitwiseExtract(dqa, 14)
  viShadowFlagsD = bitwiseExtract(dqa, 15)
  # Good data, use with confidence
  mask = viQualityFlagsS.eq(0)  \
    .And(viQualityFlagsD.eq(0)) \
    .And(viQualityFlagsS.eq(1)) \
    .And(viQualityFlagsD.eq(1)) \
    .And(viSnowIceFlagsD).eq(0); 
    # .And(viShadowFlagsD).eq(0); # No shadow
  return image.updateMask(mask)

mod13q1_QC = terra.map(modisQA_mask)
myd13q1_QC = aqua.map(modisQA_mask)

mxd13q1_cleaned = mod13q1_QC.select('EVI').merge(myd13q1_QC.select('EVI'))
mxd13q1_cleaned_sorted = mxd13q1_cleaned.sort("system:time_start")

bool_dict = {
    '0':'ocean',
    '1':'non_crop',
    '2':'crop_irrigated',
    '3':'crop_rainfed',
}
lgrip30 = ee.ImageCollection("projects/sat-io/open-datasets/GFSAD/LGRIP30").mosaic() #.clip(geoj)
crop_data = lgrip30.select('b1').gt(1).rename('crop')

## Admin

In [5]:
iso3 = 'NER'
country = 'Niger'
release_type = 'gbOpen'

adm = 'ADM0'
geo_url = f'https://www.geoboundaries.org/api/current/{release_type}/{iso3}/{adm}/'
res = requests.get(geo_url).json()
print("Reading "+ res['gjDownloadURL'])
adm0_ner = gpd.read_file(res['gjDownloadURL'])

adm = 'ADM1'
geo_url = f'https://www.geoboundaries.org/api/current/{release_type}/{iso3}/{adm}/'
res = requests.get(geo_url).json()
print("Reading "+ res['gjDownloadURL'])
adm1_ner = gpd.read_file(res['gjDownloadURL'])

adm = 'ADM2'
geo_url = f'https://www.geoboundaries.org/api/current/{release_type}/{iso3}/{adm}/'
res = requests.get(geo_url).json()
print("Reading "+ res['gjDownloadURL'])
adm2_ner = gpd.read_file(res['gjDownloadURL'])

Reading https://github.com/wmgeolab/geoBoundaries/raw/9469f09/releaseData/gbOpen/NER/ADM0/geoBoundaries-NER-ADM0.geojson
Reading https://github.com/wmgeolab/geoBoundaries/raw/9469f09/releaseData/gbOpen/NER/ADM1/geoBoundaries-NER-ADM1.geojson
Reading https://github.com/wmgeolab/geoBoundaries/raw/9469f09/releaseData/gbOpen/NER/ADM2/geoBoundaries-NER-ADM2.geojson


### Labels

In [4]:
# labels = gpd.read_file(join(expanduser('~'), 'tmp', 'usgs', 'usgs-niger.shp'))
# labels = labels.loc[labels.land_use_t==1].copy()

In [5]:
# adm2_ner.explore()

## Pull Data

In [6]:
aoi = gpd_to_gee(adm0_ner)

In [9]:
scale_factor=0.0001
# apply cropland mask to imageCollection
def cropmask(img):
  return img.clip(aoi) #.multiply(scale_factor) # unmask(-1)
mxd13q1 = mxd13q1_cleaned_sorted.map(cropmask)
# def cropmask(img):
#   return img.updateMask(crop_data).clip(aoi) #.multiply(scale_factor) # unmask(-1)

## Map

In [92]:
one = adm2_ner_sel.iloc[[0]]
centx, centy = one.centroid.x.iloc[0], one.centroid.y.iloc[0]

# Symbology
viVis = {
  'min': 0.0,
  'max': 8000.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
}
# Symbology
viVis2 = {
  'min': 0,
  'max': 1,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
}

C:\Users\WB514197\AppData\Local\Temp\ipykernel_5960\910987317.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [13]:
# m = Map(center=[centy, centx], zoom=10)
# # m.add_basemap('Esri.WorldImagery')
# m.addLayer(modis.select('EVI').first().clip(aoi), viVis, 'Raw VI')
# m.addLayer(mxd13q1.select('EVI').first(), viVis, 'VI')
# m.addLayerControl()
# m

## Load Xarray Data

In [10]:
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [11]:
utm = 'EPSG:32632' # 'EPSG:4326' 'SR-ORG:6974' 'EPSG:23847'

In [21]:
ds = xr.open_dataset(
    mxd13q1,
    engine = 'ee',
    geometry = aoi.geometry(),
    crs = utm,
    scale = 250, # 0.05 250
    )

In [24]:
ds_all = []
scale_factor=0.0001
months = range(1, 13)
for month in tqdm(months):
    modis_sel = mxd13q1.filter(ee.Filter.calendarRange(month, month, 'month'))
    # modis_sel = mxd13q1.filterDate(f'{year}-{month}-01', f'{year}-{month+1}-01')
    ds = xr.open_dataset(
        modis_sel,
        engine = 'ee',
        geometry = aoi.geometry(),
        crs = utm,
        scale = 250,
        )
    ds = ds*scale_factor
    # ds = ds.compute()
    ds_all.append(ds)
# print(f"Finished {year}")

  0%|          | 0/12 [00:00<?, ?it/s]

In [49]:
ds = xr.concat(ds_all, dim='time')

In [50]:
data_dir = join(expanduser('~'), 'data', 'niger')
# data_dir = join(expanduser('~'), 'tmp', 'niger')
os.path.exists(data_dir)

True

In [51]:
ds = ds.isel(Y=slice(None, None, -1))

In [52]:
ds.to_netcdf(join(data_dir, "modis_niger.nc"))

In [53]:
ds = xr.open_dataset(join(data_dir, "modis_niger.nc"))

In [54]:
size_in_bytes = ds.nbytes
def human_readable_size(size_in_bytes):
    # Convert size to a human-readable format
    for unit in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if size_in_bytes < 1024:
            return f"{size_in_bytes:.2f} {unit}"
        size_in_bytes /= 1024

# Get the human-readable size of the dataset
readable_size = human_readable_size(size_in_bytes)
print(f"Size of the dataset: {readable_size}")

Size of the dataset: 6.04 GB


## Dask Prep

In [56]:
# Dask Prep
workers = 30
cluster = LocalCluster(n_workers=workers)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 30
Total threads: 270,Total memory: 0.98 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43641,Workers: 30
Dashboard: http://127.0.0.1:8787/status,Total threads: 270
Started: Just now,Total memory: 0.98 TiB
Comm: tcp://127.0.0.1:37591,Total threads: 9
Dashboard: http://127.0.0.1:40169/status,Memory: 33.58 GiB
Nanny: tcp://127.0.0.1:33567,


In [58]:
# client.restart()

In [195]:
client.close()
client.shutdown()

## Phenolopy

In [60]:
dask_chunks={"time": -1, "X": 500, "Y": 500}
ds = ds.chunk(dask_chunks)
ds = ds.rename_vars({'EVI':'veg_index'})
ds = phenolopy.remove_outliers(ds=ds, method='median', user_factor=2, z_pval=0.05)
print("Finished removing outliers")
ds = ds.chunk(dask_chunks)
ds = phenolopy.interpolate(ds=ds, method='interpolate_na')
ds = phenolopy.smooth(ds=ds, method='savitsky', window_length=3, polyorder=1)
ds = ds.compute()

Outlier removal method: median with a user factor of: 2
> Generated roll window size is an even number, added 1 to make it odd (7).
> Warning: dataset contains nan values. You may want to interpolate next.
> Outlier removal successful.

Finished removing outliers
Interpolating dataset using method: interpolate_na.
> Warning: dataset still contains nan values. The first and/or last time slices may be empty.
> Interpolation successful.

Smoothing method: savitsky with window length: 3 and polyorder: 1.
> Warning: dataset contains nan values. You may want to interpolate next.
> Smoothing successful.



<string>:6: FutureWarning: 'Y' is deprecated and will be removed in a future version. Please use 'YE' instead of 'Y'.
/home/jupyter-wb514197/.conda/envs/rtmis/lib/python3.12/site-packages/tornado/ioloop.py:937: RuntimeWarning: Cannot close a netcdf_file opened with mmap=True, when netcdf_variables or arrays referring to its data still exist. All data arrays obtained from such files refer directly to data on disk, and must be copied before the file can be cleanly closed. (See netcdf_file docstring for more information on mmap.)
  val = self.callback()
/home/jupyter-wb514197/.conda/envs/rtmis/lib/python3.12/site-packages/xarray/backends/lru_cache.py:71: RuntimeWarning: Cannot close a netcdf_file opened with mmap=True, when netcdf_variables or arrays referring to its data still exist. All data arrays obtained from such files refer directly to data on disk, and must be copied before the file can be cleanly closed. (See netcdf_file docstring for more information on mmap.)
  del self._cache[

In [137]:
ds.to_netcdf(join(data_dir, "modis_niger_processed.nc"))

In [64]:
# calc phenometrics via phenolopy!
ds_phenos = phenolopy.calc_phenometrics(da=ds['veg_index'], 
                                        peak_metric='pos', 
                                        base_metric='vos', 
                                        method='seasonal_amplitude', 
                                        factor=0.2, 
                                        thresh_sides='two_sided') 
                                        # abs_value=0.1)

Initialising calculation of phenometrics.

Beginning extraction of CRS metadata.
> Extracting CRS metadata.
> No CRS metadata found. Returning None.

Beginning calculation of phenometrics. This can take awhile - please wait.

Beginning calculation of peak of season (pos) values and times.
> Calculating peak of season (pos) values.
> Calculating peak of season (pos) times.
> Success!

Beginning calculation of valley of season (vos) values and times.
> Calculating valley of season (vos) values.
> Calculating valley of season (vos) times.
> Success!

Beginning calculation of middle of season (mos) values (times not possible).
> Calculating middle of season (mos) values.
> Success!

Beginning calculation of base (bse) values (times not possible).
> Calculating base (bse) values.
> Success!

Beginning calculation of amplitude of season (aos) values (times not possible).
> Calculating amplitude of season (aos) values.
> Success!

Beginning calculation of start of season (sos) values and time

/home/jupyter-wb514197/.conda/envs/rtmis/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast


> Success!

Beginning calculation of end of season (eos) values and times.
> Calculating end of season (eos) values via method: seasonal_amplitude.
> Calculating end of season (eos) times via method: seasonal_amplitude.


/home/jupyter-wb514197/.conda/envs/rtmis/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast


> Success!

Beginning calculation of length of season (los) values (times not possible).
> Calculating length of season (los) values.
> Success!

Beginning calculation of rate of increase (roi) values (times not possible).
> Calculating rate of increase (roi) values.
> Success!

Beginning calculation of rate of decrease (rod) values (times not possible).
> Calculating rate of decrease (rod) values.
> Success!

Beginning calculation of long integral of season (lios) values (times not possible).
> Calculating long integral of season (lios) values.
> Success!

Beginning calculation of short integral of season (sios) values (times not possible).
> Calculating short integral of season (sios) values.
> Success!

Beginning calculation of long integral of total (liot) values (times not possible).
> Calculating long integral of total (liot) values.
> Success!

Beginning calculation of short integral of total (siot) values (times not possible).
> Calculating short integral of total (siot) values

In [65]:
ds_phenos

<xarray.Dataset> Size: 3GB
Dimensions:      (X: 6727, Y: 5242)
Coordinates:
  * X            (X) float64 54kB -4.661e+05 -4.658e+05 ... 1.215e+06 1.215e+06
  * Y            (Y) float64 42kB 2.618e+06 2.618e+06 ... 1.308e+06 1.308e+06
    time         (X, Y) datetime64[ns] 282MB 2023-01-01 ... 2023-01-01
Data variables: (12/18)
    pos_values   (X, Y) float32 141MB nan nan nan nan nan ... nan nan nan nan
    pos_times    (X, Y) float32 141MB nan nan nan nan nan ... nan nan nan nan
    mos_values   (X, Y) float32 141MB nan nan nan nan nan ... nan nan nan nan
    vos_values   (X, Y) float32 141MB nan nan nan nan nan ... nan nan nan nan
    vos_times    (X, Y) float32 141MB nan nan nan nan nan ... nan nan nan nan
    bse_values   (X, Y) float32 141MB nan nan nan nan nan ... nan nan nan nan
    ...           ...
    roi_values   (X, Y) float32 141MB nan nan nan nan nan ... nan nan nan nan
    rod_values   (X, Y) float32 141MB nan nan nan nan nan ... nan nan nan nan
    lios_values  (X, Y) float32 141MB nan nan nan nan nan ... nan nan nan nan
    sios_values  (X, Y) float32 141MB nan nan nan nan nan ... nan nan nan nan
    liot_values  (X, Y) float32 141MB nan nan nan nan nan ... nan nan nan nan
    siot_values  (X, Y) float32 141MB nan nan nan nan nan ... nan nan nan nan

## Get Training Data

In [87]:
labels = gpd.read_file(join(expanduser('~'), 'data', 'niger', 'sahel_training_data_20211110.geojson'))
labels.Class.value_counts()

In [90]:
adm0_ner.crs==labels.crs

True

In [92]:
labels = labels.loc[labels.intersects(adm0_ner.geometry.iloc[0])].copy()
labels.Class.value_counts()

Class
0    340
1    262
Name: count, dtype: int64

In [93]:
labels = labels.to_crs(utm)
labels.loc[:, 'geometry'] = labels.geometry.centroid

In [94]:
adm2_ner = adm2_ner.to_crs(utm)

In [100]:
res = []
for idx, row in tqdm(labels.iterrows()):
    geom = row.geometry
    x = geom.x
    y = geom.y
    # x, y = row['geometry'].coords[0]
    ds_sel = ds_phenos.sel(Y=y, X=x, method='nearest')

    s = ds_sel.to_pandas()
    s.name = idx
    s['X'] = ds_sel.X.values
    s['Y'] = ds_sel.Y.values
    s['class'] = int(row.Class)
    s['geometry'] = geom
    try:
        s['adm2'] = adm2_ner.loc[adm2_ner.geometry.contains(geom)].iloc[0]['shapeName']
    except:
        s['adm2'] = None
    df_sample = s.to_frame().transpose()

    # df = ds_sel.to_dataframe()
    # df.loc[:, 'id_sample'] = idx
    res.append(df_sample)

0it [00:00, ?it/s]

In [101]:
df_labels = pd.concat(res)

In [106]:
gdf_labels = gpd.GeoDataFrame(df_labels, geometry='geometry', crs=utm)
gdf_sel = gdf_labels.loc[gdf_labels.adm2.isna()].copy()

In [110]:
gdf_sel

,pos_values,pos_times,mos_values,vos_values,vos_times,bse_values,aos_values,sos_values,sos_times,eos_values,...,rod_values,lios_values,sios_values,liot_values,siot_values,X,Y,class,geometry,adm2
4486,0.385517,257.0,0.360789,0.141633,353.0,0.143167,0.243883,0.1852,169.0,0.1969,...,0.003368,4.95385,2.262818,9.593398,3.219896,-139833.4527382861,1349303.8511287074,0,POINT (-139749.246 1349390.147),None
6031,0.2811,233.0,0.261532,0.120867,129.0,0.1356,0.160233,0.160267,185.0,0.154733,...,0.001316,4.1406,1.844133,7.7583,2.319297,172416.5472617139,1519053.8511287074,1,POINT (172507.779 1519049.092),None


In [129]:
# import folium as flm
# m = gdf_sel[['class', 'geometry']].explore()
# adm2_ner.explore(m=m)
# flm.LayerControl().add_to(m)
# m

In [130]:
df_labels.loc[6031, "adm2"] = "Madaoua"
df_labels.loc[4486, "adm2"] = "Dosso"

## Classification Model

In [132]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [144]:
# df_labels['class']

In [139]:
df_labels.loc[:, "crop"] = df_labels['class'].astype(int)

In [141]:
df_labels.drop(columns=['class'], inplace=True)

In [142]:
df_samples = df_labels.copy()

In [167]:
df_samples.fillna(0, inplace=True)

In [182]:
df_samples.loc[df_samples.isna().any(axis=1)][['sos_values', 'sos_times', 'crop']]

,sos_values,sos_times,crop


In [183]:
df_samples.columns

Index(['pos_values', 'pos_times', 'mos_values', 'vos_values', 'vos_times',
       'bse_values', 'aos_values', 'sos_values', 'sos_times', 'eos_values',
       'eos_times', 'los_values', 'roi_values', 'rod_values', 'lios_values',
       'sios_values', 'liot_values', 'siot_values', 'X', 'Y', 'geometry',
       'adm2', 'crop'],
      dtype='object')

In [184]:
subset = ['sos_values', 'sos_times', 'eos_values', 'eos_times', 'pos_values', 'pos_times', 'los_values']

In [185]:
y = df_samples['crop']
X = df_samples.loc[:, subset]
# X = df_samples.loc[:,'pos_values':'siot_values']

In [186]:
y.value_counts()

crop
0    340
1    262
Name: count, dtype: int64

In [187]:
X.isna().sum()

sos_values    0
sos_times     0
eos_values    0
eos_times     0
pos_values    0
pos_times     0
los_values    0
dtype: int64

In [188]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 100, stratify=df_samples.loc[:,('crop')])

In [189]:
rf_pipeline = Pipeline([
    ('rf', RandomForestClassifier())
    ])
rf_params = {
          'rf__n_estimators': (100,1000,5000),
          'rf__criterion': ('gini','entropy'),
          'rf__max_depth': (1,3,5),
          'rf__min_samples_split': (2,5,10),
          'rf__random_state': [10]
         }
k = 2

rf_grid_model = GridSearchCV(
    estimator=rf_pipeline,
    param_grid=rf_params,
    cv=k,
    scoring=['accuracy','precision','recall'],
    refit='accuracy' 
)

crop_fitted = rf_grid_model.fit(X_train, y_train.ravel())
crop_results = crop_fitted.cv_results_
crop_results = pd.DataFrame(crop_results)

In [190]:
params = crop_results.sort_values('mean_test_accuracy', ascending=False).iloc[0]['params']
params

{'rf__criterion': 'gini',
 'rf__max_depth': 5,
 'rf__min_samples_split': 5,
 'rf__n_estimators': 100,
 'rf__random_state': 10}

In [192]:
criterion = params['rf__criterion']
max_depth = params['rf__max_depth']
min_samples_split = params['rf__min_samples_split']
n_estimators = params['rf__n_estimators']
random_state = params['rf__random_state']

In [193]:
rf_best = RandomForestClassifier(criterion = criterion, max_depth = max_depth, min_samples_split = min_samples_split, n_estimators = n_estimators, random_state=random_state)
rf_best_fit = rf_best.fit(X_train, y_train.ravel())

In [194]:
# performing predictions on the test dataset
y_pred = rf_best_fit.predict(X_test)
y_train_pred = rf_best_fit.predict(X_train)

# using metrics module for accuracy calculation
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))
print("R-Squared OF THE MODEL: ", metrics.r2_score(y_test, y_pred))

ACCURACY OF THE MODEL:  0.8016528925619835
R-Squared OF THE MODEL:  0.1942286348501664


In [178]:
# performing predictions on the test dataset
y_pred = rf_best_fit.predict(X_test)
y_train_pred = rf_best_fit.predict(X_train)

# using metrics module for accuracy calculation
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))
print("R-Squared OF THE MODEL: ", metrics.r2_score(y_test, y_pred))

ACCURACY OF THE MODEL:  0.8099173553719008
R-Squared OF THE MODEL:  0.2278024417314094


In [179]:
# using metrics module for accuracy calculation
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_train, y_train_pred))
print("R-Squared OF THE MODEL: ", metrics.r2_score(y_train, y_train_pred))

ACCURACY OF THE MODEL:  0.9272349272349273
R-Squared OF THE MODEL:  0.703859414579229
